[View in Colaboratory](https://colab.research.google.com/github/nikhilsingh291/visual_q_ans/blob/master/NIKHIL_SINGH_Session7.ipynb)

#Deep Learning for Visual Question Answering


![alt text](https://avisingh599.github.io/images/vqa/sample_results.jpg)

This is about answering general english questions based on images by deep learning.

I will be using LSTM+CNN approach to solve this problem and get a better accuracy.

In [8]:
!mkdir data
%cd data


/content/data


In [9]:
# Downloads the training and validation sets from visualqa.org. 

!wget http://visualqa.org/data/mscoco/vqa/Questions_Train_mscoco.zip
!wget http://visualqa.org/data/mscoco/vqa/Questions_Val_mscoco.zip
!wget http://visualqa.org/data/mscoco/vqa/Annotations_Train_mscoco.zip
!wget http://visualqa.org/data/mscoco/vqa/Annotations_Val_mscoco.zip

!unzip \*.zip

--2018-06-04 17:30:16--  http://visualqa.org/data/mscoco/vqa/Questions_Train_mscoco.zip
Resolving visualqa.org (visualqa.org)... 128.173.88.40
Connecting to visualqa.org (visualqa.org)|128.173.88.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21985607 (21M) [application/zip]
Saving to: ‘Questions_Train_mscoco.zip’

Questions_Train_msc 100%[===================>]  20.97M  23.5MB/s    in 0.9s    

2018-06-04 17:30:17 (23.5 MB/s) - ‘Questions_Train_mscoco.zip’ saved [21985607/21985607]

--2018-06-04 17:30:18--  http://visualqa.org/data/mscoco/vqa/Questions_Val_mscoco.zip
Resolving visualqa.org (visualqa.org)... 128.173.88.40
Connecting to visualqa.org (visualqa.org)|128.173.88.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10594497 (10M) [application/zip]
Saving to: ‘Questions_Val_mscoco.zip’

Questions_Val_mscoc 100%[===================>]  10.10M  14.4MB/s    in 0.7s    

2018-06-04 17:30:18 (14.4 MB/s) - ‘Questions_Val_mscoco.zip’

In [13]:
!ls
%cd ..
!ls
!mkdir features

Annotations_Train_mscoco.zip
Annotations_Val_mscoco.zip
features
mscoco_train2014_annotations.json
mscoco_val2014_annotations.json
MultipleChoice_mscoco_train2014_questions.json
MultipleChoice_mscoco_val2014_questions.json
OpenEnded_mscoco_train2014_questions.json
OpenEnded_mscoco_val2014_questions.json
Questions_Train_mscoco.zip
Questions_Val_mscoco.zip
/content
data  datalab


In [14]:
%cd features
!ls
# Downloads and unzips the VGG features computed on the COCO dataset. 

!wget http://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
!unzip coco.zip -d .

/content/features
--2018-06-04 17:34:34--  http://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip [following]
--2018-06-04 17:34:34--  https://cs.stanford.edu/people/karpathy/deepimagesent/coco.zip
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 764830639 (729M) [application/zip]
Saving to: ‘coco.zip’

coco.zip             91%[=================>  ] 668.34M  14.4MB/s    eta 3s     

coco.zip            100%[===================>] 729.40M  15.7MB/s    in 30s     

2018-06-04 17:35:04 (24.4 MB/s) - ‘coco.zip’ saved [764830639/764830639]

Archive:  coco.zip
  inflating: ./coco/dataset.json     
  inflating: ./coco/readme.txt       
  inflating: ./coco/vgg_feats.mat    


In [15]:
!ls

coco  coco.zip


In [0]:
!pwd
!ls
!pip install spacy

/content/features
coco  coco.zip
    100% |████████████████████████████████| 17.6MB 1.8MB/s 
    100% |████████████████████████████████| 92kB 19.1MB/s 
    100% |████████████████████████████████| 1.2MB 19.4MB/s 
    100% |████████████████████████████████| 51kB 16.8MB/s 
    100% |████████████████████████████████| 194kB 23.9MB/s 
    100% |████████████████████████████████| 71kB 19.2MB/s 
    100% |████████████████████████████████| 604kB 19.6MB/s 
    100% |████████████████████████████████| 51kB 6.4MB/s 
    100% |████████████████████████████████| 389kB 23.1MB/s 
    100% |████████████████████████████████| 143kB 22.2MB/s 
  Running setup.py bdist_wheel for spacy ... - \ | /

 - \ | / - \ | / - \ | / -

In [17]:
!python3 -m spacy download en

    100% |████████████████████████████████| 37.4MB 71.4MB/s 
  Running setup.py install for en-core-web-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
import operator
from itertools import zip_longest
from collections import defaultdict

def selectFrequentAnswers(questions_train, answers_train, images_train, maxAnswers):
	answer_fq= defaultdict(int)
	#build a dictionary of answers
	for answer in answers_train:
		answer_fq[answer] += 1

	sorted_fq = sorted(answer_fq.items(), key=operator.itemgetter(1), reverse=True)[0:maxAnswers]
	top_answers, top_fq = zip(*sorted_fq)
	new_answers_train=[]
	new_questions_train=[]
	new_images_train=[]
	#only those answer which appear int he top 1K are used for training
	for answer,question,image in zip(answers_train, questions_train, images_train):
		if answer in top_answers:
			new_answers_train.append(answer)
			new_questions_train.append(question)
			new_images_train.append(image)

	return (new_questions_train,new_answers_train,new_images_train)

def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return izip_longest(*args, fillvalue=fillvalue)

In [0]:
import numpy as np
from keras.utils import np_utils


def get_questions_tensor_timeseries(questions, nlp, timesteps):
	'''
	Returns a time series of word vectors for tokens in the question

	Input:
	questions: list of unicode objects
	nlp: an instance of the class English() from spacy.en
	timesteps: the number of 

	Output:
	A numpy ndarray of shape: (nb_samples, timesteps, word_vec_dim)
	'''
	assert not isinstance(questions, basestring)
	nb_samples = len(questions)
	word_vec_dim = nlp(questions[0])[0].vector.shape[0]
	questions_tensor = np.zeros((nb_samples, timesteps, word_vec_dim))
	for i in xrange(len(questions)):
		tokens = nlp(questions[i])
		for j in xrange(len(tokens)):
			if j<timesteps:
				questions_tensor[i,j,:] = tokens[j].vector

	return questions_tensor

def get_questions_matrix_sum(questions, nlp):
	'''
	Sums the word vectors of all the tokens in a question
	
	Input:
	questions: list of unicode objects
	nlp: an instance of the class English() from spacy.en

	Output:
	A numpy array of shape: (nb_samples, word_vec_dim)	
	'''
	assert not isinstance(questions, basestring)
	nb_samples = len(questions)
	word_vec_dim = nlp(questions[0])[0].vector.shape[0]
	questions_matrix = np.zeros((nb_samples, word_vec_dim))
	for i in xrange(len(questions)):
		tokens = nlp(questions[i])
		for j in xrange(len(tokens)):
			questions_matrix[i,:] += tokens[j].vector

	return questions_matrix

def get_answers_matrix(answers, encoder):
	'''
	Converts string objects to class labels

	Input:
	answers:	a list of unicode objects
	encoder:	a scikit-learn LabelEncoder object

	Output:
	A numpy array of shape (nb_samples, nb_classes)
	'''
	assert not isinstance(answers, basestring)
	y = encoder.transform(answers) #string to numerical class
	nb_classes = encoder.classes_.shape[0]
	Y = np_utils.to_categorical(y, nb_classes)
	return Y

def get_images_matrix(img_coco_ids, img_map, VGGfeatures):
	'''
	Gets the 4096-dimensional CNN features for the given COCO
	images
	
	Input:
	img_coco_ids: 	A list of strings, each string corresponding to
				  	the MS COCO Id of the relevant image
	img_map: 		A dictionary that maps the COCO Ids to their indexes 
					in the pre-computed VGG features matrix
	VGGfeatures: 	A numpy array of shape (nb_dimensions,nb_images)

	Ouput:
	A numpy matrix of size (nb_samples, nb_dimensions)
	'''
	assert not isinstance(img_coco_ids, basestring)
	nb_samples = len(img_coco_ids)
	nb_dimensions = VGGfeatures.shape[0]
	image_matrix = np.zeros((nb_samples, nb_dimensions))
	for j in xrange(len(img_coco_ids)):
		image_matrix[j,:] = VGGfeatures[:,img_map[img_coco_ids[j]]]

	return image_matrix

In [0]:
import numpy as np
import scipy.io
import sys

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Reshape
from keras.layers import Merge
from keras.layers.recurrent import LSTM
from keras.utils import np_utils, generic_utils
from keras.callbacks import ModelCheckpoint, RemoteMonitor

from sklearn.externals import joblib
from sklearn import preprocessing

#from spacy.en import English

#from utils import grouper, selectFrequentAnswers
#from features import get_images_matrix, get_answers_matrix, get_questions_tensor_timeseries

In [7]:
!ls


datalab


In [27]:
word_vec_dim= 300
img_dim = 4096
max_len = 30
nb_classes = 1000

questions_train = open('../data/preprocessed/questions_train2014.txt', 'r').read().decode('utf8').splitlines()
questions_lengths_train = open('../data/preprocessed/questions_lengths_train2014.txt', 'r').read().decode('utf8').splitlines()
answers_train = open('../data/preprocessed/answers_train2014_modal.txt', 'r').read().decode('utf8').splitlines()
images_train = open('../data/preprocessed/images_train2014.txt', 'r').read().decode('utf8').splitlines()
vgg_model_path = '../features/coco/vgg_feats.mat'

FileNotFoundError: ignored